In [2]:
import geopandas as gpd
from geopandas import GeoDataFrame
import os
import zipfile
import pandas as pd
import glob
import zipfile
from pandas import DataFrame
layer_crs= 'epsg:2039'
project_folder = os.path.dirname(os.getcwd())

In [2]:
# The required files' paths
trees = glob.glob(project_folder + r'\TelAvivLayers\elements_as_length\Trees\*[!.zip]\*.shp')
lighting = glob.glob(project_folder + r'\TelAvivLayers\elements_as_length\lighting\*[!.zip]\*.shp')
bus_stations = glob.glob(project_folder + r'\TelAvivLayers\elements_as_length\bus_stations\*.shp')

In [11]:

# Organize all paths and names in one dictionary and save more information if necessary (for example, the heights of the building)
data_dic = {'trees':trees,'lighting':lighting,'bus_stations':bus_stations}

In [12]:
# Upload and then merge all files for each item in the dictionary
for files in data_dic.items():
    data_list = []
    name = files[0]
    print('run on {}'.format(name))
    # In the new file, which data should be retained?
    cols = ['geometry']
    files_path =  files[1]
    for file_path in files_path:
        file =  gpd.read_file(file_path)[cols].reset_index()
        # Make sure the file is in the correct CRS and if not, transform it to the right one
        if file .crs.srs !=layer_crs:
            print('different crs')
            file.to_crs(layer_crs)
        file_name =  file_path.split('\\')[-2]
        print('in {} has {} {} '.format(file_name ,len(file),name))
        file['name'] = file_name
        data_list.append(file)
    # Combine all and leave only one among those  with the same geometry
    one_file = pd.concat(data_list)
    print('there is  {} {}'.format(len(one_file),name))
    clean_file  = one_file.drop_duplicates(subset='geometry')
    print('there is  {} {} after dropping duplicate'.format(len(clean_file),name))
    clean_file.to_file('output/streets_elements/elements/elements_as_length/{}.shp'.format(name))

run on synagogues
in synagogues has 481 synagogues 
there is  481 synagogues
there is  443 synagogues after dropping duplicate


## Add streets' elements as attributes to the street

In [4]:
res_path = 'output/streets_elements'
clean_network = gpd.read_file(r'{}/clean_network.shp'.format(res_path))
col_to_leave = ['length','geometry','oidrechov']
clean_network['length'] = clean_network.length

In [5]:
objects_folder = glob.glob('output/streets_elements//elements/elements_as_length/*.shp')
objects_folder

['output/streets_elements//elements/elements_as_length\\bus_stations.shp',
 'output/streets_elements//elements/elements_as_length\\lighting.shp',
 'output/streets_elements//elements/elements_as_length\\trees.shp']

### Relative Count

In [6]:
files_names = ['trees','lighting','bus_stations']
for objects_file in objects_folder:
    name = objects_file.split("\\")[-1].split('.')[0]
    print(name)
    if  not name in files_names:
        print("{} not in files name")
        continue
    clean_network_temp =clean_network[col_to_leave]
    objects_file = gpd.read_file(objects_file)
    if 'index' in objects_file.columns:
        objects_file = objects_file.drop(columns=['index'])
    # Find the nearest street segment for each object
    sjoin= objects_file.sjoin_nearest(clean_network_temp,distance_col='dis', max_distance=50)
    path = 'output/streets_elements/detailed_folder/elements_as_length/{}'.format(name)
    os.makedirs(path,exist_ok = True)
    sjoin.to_file('{}/sjoin.shp'.format(path))
    # Aggregate objects that share the same closet street
    dissolve = sjoin.dissolve(by = 'oidrechov',aggfunc='count')
    clean_network_temp.set_index('oidrechov', inplace=True)
    clean_network.set_index('oidrechov', inplace=True)
    # Each street segment is updated with the number of objects around it and the ratio between that number and its length.
    clean_network_temp['count'] = dissolve['length']
    clean_network_temp['ratio'] = clean_network_temp['count'] / clean_network_temp['length']
    # Results are stored in two files - one specifically for the current object and one for the general streets network.
    clean_network[name] = clean_network_temp['ratio']
    clean_network.reset_index(inplace=True)
    clean_network_temp.to_file('{}/streets_elements_{}.shp'.format(path,name))
clean_network.to_file('output/streets_elements/streets_elements.shp')
print('finish')

bus_stations


C:\Users\Achituv\AppData\Local\Temp\ipykernel_19564\5065397.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sjoin.to_file('{}/sjoin.shp'.format(path))
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

lighting


C:\Users\Achituv\AppData\Local\Temp\ipykernel_19564\5065397.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sjoin.to_file('{}/sjoin.shp'.format(path))
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

trees


C:\Users\Achituv\AppData\Local\Temp\ipykernel_19564\5065397.py:16: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sjoin.to_file('{}/sjoin.shp'.format(path))
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

finish
